In [1]:
import os
import math
import datetime
import pandas as pd

def calculate_h_index(quotation): #h_ndex
    quotation.sort(reverse=True)  # 논문을 내림차순으로 정렬

    h_index = 0
    for i, quotation in enumerate(quotation):
        if quotation >= i + 1:
            h_index = i + 1
        else:
            break

    return h_index

def calculate_i10_index(quotation):  #i10_index
    i10_index = sum(1 for quotation in quotation if quotation >= 10)
    return i10_index

def calculate_g_index(quotation):  #g_index
    quotation.sort(reverse=True)  # 논문을 내림차순으로 정렬

    g_index = 0
    quotation_sum=0
    for i, quotation in enumerate(quotation):
        quotation_sum+=quotation
        if quotation_sum >= (i + 1)**2:
            g_index = i + 1

    return g_index

#Object 타입에서 숫자형 체크용
def is_numeric(value):
    try:
        float(value)  # float으로 변환 가능한 경우 숫자
        return True
    except (ValueError, TypeError):
        return False

def recentness(pYears):
    # dt = datetime.datetime.now()#현재날짜
    dt = 2013
    rct = 0
    lct = 0
    if len(pYears) == 0:
        rct = 0
        lct = 0
    else:
        year_avg = sum(pYears) / len(pYears)
        if year_avg >= int(dt)-2: #int(dt.year)-2:
            lct = 1
        # else:
        #     lct = max(round((1-(int(dt)-3-year_avg)*0.1),2), 0)
        elif int(dt)-15 < year_avg <= int(dt)-3:
            lct = max(round((1-(int(dt)-3-year_avg)*0.1),2), 0)

        rct += len(pYears) #연도 개수

    return rct, lct

In [8]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

client = MongoClient('mongodb://10.255.92.141:27017', authSource='admin')

# keyid = 2157 #input
# idTest = client['ID']['test'] #Domestic
# publicNewfactor = client['PUBLIC']['new_factor']
# idTest_keyid = idTest.find({ 'keyId' : keyid })
# maxFactor = publicNewfactor.find_one({'keyId' : keyid})

# a = idTest.count_documents({ 'keyId' : keyid })
# print(a)

keyid2=2152
idTest = client['ID']['test'] #Domestic
idInter_keyid = idTest.find({ 'keyId' : keyid2 })
publicNewfactor_inter = client['PUBLIC']['new_factor_inter']
maxFactor_inter = publicNewfactor_inter.find_one({'keyId' : keyid2})

b = idTest.count_documents({ 'keyId' : keyid2 })
print(b)

3862


In [9]:
sites = ["WOS"]

sco=[]
h_idx=[]
i10_idx=[]
g_idx=[]

df = pd.DataFrame({
    'score': sco,
    'h_index': h_idx,
    'i10_index': i10_idx,
    'g_index': g_idx
})

for i in idInter_keyid:
    citations = []
    years = []
    for site in sites:
        if site in i.keys():
            papersIds = i[site]["papers"]

            for paper in papersIds:
                idSite = client[site]['Rawdata'] 
                idSite_pId = idSite.find_one({ '_id' : ObjectId(str(paper)) })

                citations.append(int(idSite_pId["citation"]))
                years.append(int(idSite_pId["issue_year"]))

            h_index = calculate_h_index(citations)
            i10_index = calculate_i10_index(citations)
            g_index = calculate_g_index(citations)


    lct = i['factor']['lct'] #생산성
    rct = i['factor']['rct'] #생산성
    acc = i['factor']['acc'] #주제적합도
    qual = i['factor']['interQual'] #품질
    norm_scoquality = qual/maxFactor_inter['interQual'] #정규화된 품질
    norm_recentness = ((rct/maxFactor_inter['recentness'])+lct)*0.66 #정규화된 생산성

    score = (norm_scoquality + norm_recentness + acc)/3*100
    # print(f'citations: {citations}, years: {years}')

    new_data={'score': score, 'h_index': h_index, 'i10_index': i10_index, 'g_index':g_index}

    df = pd.concat([df, pd.DataFrame(new_data, index=[0])], ignore_index=True)

df = df.sort_values(by='g_index', ascending=False)
print(df)

          score  h_index  i10_index  g_index
3535  86.760301     13.0       13.0     13.0
2833  77.415406     12.0       12.0     12.0
1181  73.055069      9.0        9.0      9.0
3736  60.763753      8.0        7.0      8.0
3101  68.794440      8.0        8.0      8.0
916   57.772694      7.0        7.0      7.0
1941  67.979502      7.0        6.0      7.0
717   55.206382      6.0        6.0      6.0
674   59.910882      6.0        6.0      6.0
2940  67.837710      6.0        6.0      6.0
1217  52.501216      6.0        6.0      6.0
2734  48.615312      5.0        5.0      5.0
2574  55.735979      5.0        5.0      5.0
2725  55.330432      5.0        5.0      5.0
47    56.296217      5.0        5.0      5.0
3490  48.155962      5.0        5.0      5.0
2017  61.187227      5.0        5.0      5.0
644   55.518367      5.0        5.0      5.0
641   52.599020      5.0        5.0      5.0
2924  56.715596      5.0        5.0      5.0
2087  54.441916      5.0        5.0      5.0
3613  55.6

In [ ]:


max_scoquality = 62888.30906843457
max_rct = 15106

max_lct = []

df_a_copy = pd.DataFrame()

# df_a_copy의 각 행에서 1번 열의 값을 추출
for index, row in df_a_copy.iterrows(): # 저자로 교체
    name = row[5]  # 1번 열의 값

    # 각 줄을 나누어서 리스트로 저장
    line_lists = [line.strip().split('|') for line in lines] # 몽고디비 데이터로 교체, 또는 삭제
    
    #빈 리스트 생성
    citations = [] #인용수
    years = [] #연도
    scoquality = 0 #품질
    for line_list in line_lists: #논문으로 교체
        if len(line_list) > 0:
            citations.append(int(line_list[0]))
            years.append(int(line_list[1]))
            scoquality += math.log(int(line_list[0]*1)+1)
    
    # print(name, years)
    rct, lct = recentness(years)
    # max_lct.append(rct)
    norm_scoquality = scoquality/max_scoquality
    # norm_recentness = rct/max_rct*0.66*lct
    norm_recentness = ((rct/max_rct)+lct)*0.66
    score = (norm_scoquality + norm_recentness)*50
    # print(rct, scoquality, scoquality/max_scoquality, rct/max_rct, lct, norm_recentness, score)
    #각 index 계산 후 df_a_copy에 새로운 열로 추가
    df_a_copy.at[index,"Paper Count"] = len(citations)
    df_a_copy.at[index,"h-index"] = calculate_h_index(citations)
    df_a_copy.at[index,"i10-index"] = calculate_i10_index(citations)
    df_a_copy.at[index,"g-index"] = calculate_g_index(citations)
    df_a_copy.at[index,"scoquality"] = norm_scoquality
    df_a_copy.at[index,"recentness"] = norm_recentness
    df_a_copy.at[index,"score"] = score
        
df_a_copy['ID'] = 'id_' + df_a_copy['ID']

# print(df_a_copy)
df_a_copy.head(20)
# df_a_copy.to_csv('output.txt', sep = ';', index=False)
# df_a_copy.to_csv('output_index.txt', index=True)
df_a_copy.to_csv('output.csv', index=False)
df_a_copy.to_csv('output_index.csv', index=True)